<a href="https://colab.research.google.com/github/TigeressCindy/The-Complete-Neural-Networks-Bootcamp-Theory-Applications/blob/master/NLP_k_Disaster_Tweets_HHC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!pip install tensorflow

In [3]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tigeress123","key":"08176849529cecd089d08991cc982be6"}'}

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:

!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 116MB/s]


In [8]:
!unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preprocess the text data
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['text'])

X_train_sequences = tokenizer.texts_to_sequences(train_data['text'])
X_test_sequences = tokenizer.texts_to_sequences(test_data['text'])

max_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

y_train = train_data['target'].values

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Build the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - accuracy: 0.6345 - loss: 0.6234 - val_accuracy: 0.7971 - val_loss: 0.4708
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 18s 65ms/step - accuracy: 0.8743 - loss: 0.3125 - val_accuracy: 0.7800 - val_loss: 0.4826
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9424 - loss: 0.1763 - val_accuracy: 0.7820 - val_loss: 0.5383
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.9672 - loss: 0.1030 - val_accuracy: 0.7695 - val_loss: 0.6374
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.9727 - loss: 0.0899 - val_accuracy: 0.7649 - val_loss: 0.6890


In [12]:
# Predict on the test set
test_predictions = model.predict(X_test_padded)
test_predictions = (test_predictions > 0.5).astype(int).flatten()

# Prepare the submission file
submission = pd.DataFrame({'id': test_data['id'], 'target': test_predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step
Submission file created successfully!


In [13]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>